# Test notebook to structure the data

In [1]:
import pandas as pd

In [2]:
data_path = "sample_data.txt"
df = pd.read_csv(data_path, sep="\t")

In [3]:
df

,onset,duration,trial_period,trial,TR_relativeToTrialOnset,trial_type,response_time,button_pressed,choice,outcome,gamble_position,rightbutton_position,total_gain
0,12.0008,2,prechoice,1,1,symmetric,NaN,NaN,NaN,NaN,bottom,NaN,NaN
1,14.0008,4,choice,1,2,symmetric,2.3919,left,gamble,NaN,bottom,top,NaN
2,18.0008,2,outcome,1,4,symmetric,NaN,NaN,NaN,3.05,NaN,NaN,19.15
3,20.0008,2,ITI,1,5,symmetric,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,22.0008,2,ITI,1,6,symmetric,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,864.0001,2,ITI,71,6,positiveSkew,NaN,NaN,NaN,NaN,NaN,NaN,NaN
356,866.0001,2,prechoice,72,1,positiveSkew,NaN,NaN,NaN,NaN,bottom,NaN,NaN
357,868.0001,4,choice,72,2,positiveSkew,2.0084,right,safe,NaN,bottom,top,NaN
358,872.0001,2,outcome,72,4,positiveSkew,NaN,NaN,NaN,0.00,NaN,NaN,18.30


# Organize: one row per trial with stimulus, behavior, and feedback columns

In [4]:
# Replace 'n/a' strings with NaN
df = df.replace("n/a", float("nan"))

# Filter by trial_period
prechoice = df[df["trial_period"] == "prechoice"][["trial", "trial_type", "gamble_position"]].copy()
choice = df[df["trial_period"] == "choice"][["trial", "response_time", "choice"]].copy()
outcome = df[df["trial_period"] == "outcome"][["trial", "outcome", "total_gain"]].copy()

# Merge into one row per trial
trials = prechoice.merge(choice, on="trial").merge(outcome, on="trial")

# Map trial_type to gain/loss values
gain_loss_map = {
    "symmetric": (3.05, 3.05),
    "negativeSkew": (1.75, 5.25),
    "positiveSkew": (5.25, 1.75),
}
trials["gain"] = trials["trial_type"].map(lambda t: gain_loss_map[t][0])
trials["loss"] = trials["trial_type"].map(lambda t: gain_loss_map[t][1])

# Select and order final columns
trials = trials[["trial", "trial_type", "gain", "loss", "response_time", "choice", "outcome", "total_gain"]]

print(f"Shape: {trials.shape}")
print(f"Unique trials: {trials['trial'].nunique()}")
print(f"Duplicated trials: {trials['trial'].duplicated().sum()}")
trials

Shape: (72, 8)
Unique trials: 72
Duplicated trials: 0


,trial,trial_type,gain,loss,response_time,choice,outcome,total_gain
0,1,symmetric,3.05,3.05,2.39190,gamble,3.05,19.15
1,2,symmetric,3.05,3.05,1.69130,gamble,-3.05,16.10
2,3,symmetric,3.05,3.05,2.32580,gamble,3.05,19.15
3,4,negativeSkew,1.75,5.25,1.47430,gamble,1.75,20.90
4,5,negativeSkew,1.75,5.25,1.05690,gamble,1.75,22.65
...,...,...,...,...,...,...,...,...
67,68,negativeSkew,1.75,5.25,1.55770,safe,0.00,16.55
68,69,negativeSkew,1.75,5.25,0.90671,gamble,1.75,18.30
69,70,symmetric,3.05,3.05,1.19050,safe,0.00,18.30
70,71,positiveSkew,5.25,1.75,0.94033,safe,0.00,18.30
